Installing required packages

In [ ]:
!pip install dvc

In [26]:
!pip install dvclive


[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!git init

!dvc init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /Users/loonycorn/projects/dvc/dvc_churn_prediction_hyperparameters/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/ana

In [3]:
!git config --global user.name "loonytest"  
!git config --global user.email "loony.test.001@gmail.com"

!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	data/
	demo-04-ChurnPrediction_HyperparameterTuningAndModelServing.ipynb



In [ ]:
!git remote add -f origin https://github.com/loonybq/dvc_churn_prediction_hyperparameters.git
    
!git branch -M main

In [7]:
!mkdir /tmp/dvc_storage_cphytuning

!dvc remote add -d localremote /tmp/dvc_storage_cphytuning

!dvc remote list

!git add .dvc/config

!git status

mkdir: /tmp/dvc_storage_cphytuning: File exists
Setting 'localremote' as a default remote.
ERROR: configuration error - config file error: remote 'localremote' already exists. Use `-f|--force` to overwrite it.
localremote	/tmp/dvc_storage_cphytuning
On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	data/
	demo-04-ChurnPrediction_HyperparameterTuningAndModelServing.ipynb



In [11]:
!git commit -m "Initialized DVC for churn prediction hyperparameter tuning"

!git push -u origin main

!git log

On branch main
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	data/
	demo-04-ChurnPrediction_HyperparameterTuningAndModelServing.ipynb

nothing added to commit but untracked files present (use "git add" to track)
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 4 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 583 bytes | 583.00 KiB/s, done.
Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/loonybq/dvc_churn_prediction_hyperparameters.git
 * [new branch]      main -> main
branch 'main' set up to track 'origin/main'.
commit c9b3e39c14d215dff5534c8ee73d13a469904e76 (HEAD -> main, origin/main)
Author: loonytest <loony.test.001@gmail.com>
Date:   Sat Jul 22 07:27:26 2023 +0530

    Initialized DVC for churn prediction hyperparameter tuning


# TODO Recording

- Go to the Github repository https://github.com/loonytest/dvc_churn_prediction_hyperparameters/tree/main and show that the commits are here
- Open Iterative Studio on a new tab
- Add a new project connected to this repo
- Come back to this notebook and continue

In [19]:
!dvc config --global studio.token isat_3Ak630M1uyW8zHqg0vyhjJH9vLv0i7Ug1TxTgzxAGTEhmph8m

In [1]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
travel_churn_data = pd.read_csv('data/CustomerTravel.csv')

travel_churn_data.head()

,Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot,Target
0,34,No,Middle Income,6,No,Yes,0
1,34,Yes,Low Income,5,Yes,No,1
2,37,No,Middle Income,3,Yes,No,0
3,30,No,Middle Income,2,No,No,0
4,30,No,Low Income,1,No,No,0


In [3]:
travel_churn_data = travel_churn_data.drop_duplicates()

travel_churn_data.shape

(447, 7)

In [4]:
mapper = {'Low Income': 0,'Middle Income': 1, 'High Income': 2}

travel_churn_data['AnnualIncomeClass'] = travel_churn_data['AnnualIncomeClass'].replace(mapper)

In [5]:
categorical_features = [
    'FrequentFlyer',
    'AccountSyncedToSocialMedia', 
    'BookedHotelOrNot'
]

categorical_transformer = Pipeline(
    steps=[('encoder', OneHotEncoder(handle_unknown = 'ignore', drop = 'first'))]
)

preprocessor = ColumnTransformer(
    transformers = [('cat_tr', categorical_transformer, categorical_features)], 
    remainder = StandardScaler()
)

In [6]:
X = travel_churn_data.drop(labels = ['Target'], axis = 1)
 
y = travel_churn_data['Target']
 
from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 124)

We will apply hyperparameter tuning on XGB Classifier to get the parameters of best performing model

### Notes:

Optuna is an open-source Python framework designed for hyperparameter optimization (HPO) in machine learning models. It helps automate the process of finding the best set of hyperparameters for a given machine learning model, which can significantly improve the model's performance.

Optuna uses a technique called "Sequential Model-Based Optimization" (SMBO) to efficiently explore the hyperparameter search space and quickly converge to a good set of hyperparameters. It employs different optimization algorithms, such as TPE (Tree-structured Parzen Estimator) and CMA-ES (Covariance Matrix Adaptation Evolution Strategy), to balance exploration and exploitation of the hyperparameter space.

Easy integration: Optuna can be easily integrated into existing machine learning code with just a few lines of code, making it straightforward to incorporate hyperparameter optimization into your projects.

Flexibility: It supports various machine learning libraries and frameworks, allowing you to optimize hyperparameters for different types of models, such as scikit-learn, TensorFlow, PyTorch, XGBoost, LightGBM, and more.

In [ ]:
pip install optuna

In [ ]:
pip install xgboost

 We’ll use Optuna to tune the hyperparameters of the XGBoost model. We’ll start by creating an objective function, which will be passed to the study.optimize function. The objective function will take the trial parameter, which is an instance of the Trial class, and will return the f1 score.
 
 Reference link-
 https://practicaldatascience.co.uk/machine-learning/how-to-use-optuna-for-xgboost-hyperparameter-tuning
 
In Optuna, the objective function is a user-defined function that represents the task you want to optimize. It encapsulates the machine learning model training, evaluation, and the metric you want to maximize or minimize. The purpose of the objective function is to take a set of hyperparameters as input, train a model with those hyperparameters, and return a value that represents the performance of the model.

A Trial object corresponds to a single execution of the objective function and is internally instantiated upon each invocation of the function.

The Trials class provides several "suggest" APIs that allow you to specify the search space for hyperparameters, allowing Optuna's optimization algorithms to explore and sample values from this space effectively.

Here are some commonly used "suggest" APIs in the Trials class:

suggest_float(name, low, high, step=None, log=False): Suggest a floating-point value for the specified hyperparameter. The value will be sampled uniformly from the range [low, high]. If step is provided, it specifies the discretization step for the sampling. If log=True, the sampling will be done in the log space.

In [17]:
from xgboost.sklearn import XGBClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 15, log = True),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0, log = True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500, log = True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10, log = True),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log = True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0,log = True),
    }

    xgb_model = XGBClassifier(**params)
    
    pipe_xgb = Pipeline(steps = [('preprocessor', preprocessor), ('classifier', xgb_model )])
    pipe_xgb.fit(X_train, y_train)

    predictions = pipe_xgb.predict(X_test)
    test_f1_score = f1_score(y_test, predictions)

    return test_f1_score

### Notes:

Next we need to define an Optuna study. We’ll set the direction to maximize, as we want to maximize the f1 score.

Sampler using TPE (Tree-structured Parzen Estimator) algorithm.

On each trial, for each parameter, TPE fits one Gaussian Mixture Model (GMM) l(x) to the set of parameter values associated with the best objective values, and another GMM g(x) to the remaining parameter values. It chooses the parameter value x that maximizes the ratio l(x)/g(x).

TPE is a Bayesian optimization algorithm that aims to efficiently find the optimal set of hyperparameters by modeling the relationship between hyperparameters and the objective function's performance. It balances exploration and exploitation to efficiently explore the hyperparameter search space and converge towards the best hyperparameter configuration.

The TPE algorithm consists of two main components: the "prior" and the "posterior" model.

Prior Model:

The prior model captures the probability distribution of the hyperparameters based on the previous observations. It is represented as two separate distributions: one for successful trials (observations that led to better results) and one for failed trials (observations that did not lead to improved results).

Posterior Model:

The posterior model represents the objective function's performance given the hyperparameters.

The posterior model provides an estimate of the probability that a specific set of hyperparameters will lead to a better objective value.


In [18]:
import optuna

study = optuna.create_study(direction = 'maximize')

print(f"Sampler is {study.sampler.__class__.__name__}")

[I 2023-07-22 08:01:27,989] A new study created in memory with name: no-name-0da54fad-80dc-4c3d-8354-3ba2afc198ab


Sampler is TPESampler


### Notes:

Finally, we can run the objective function using the study.optimize function. We’ll set the n_trials parameter to 32, which means that Optuna will run the objective function 32 times, and will try to find the best hyperparameters.

If you’ve ever used GridSearchCV for hyperparameter tuning, you’ll know that it can take a long time to run, especially if you have a large number of hyperparameters to tune. Optuna is much faster, as it uses Bayesian optimization to find the best hyperparameters.

There are warnings about untracked files here which we can ignore. If you look carefully at the output you will see the results of each trial e.g.

Trial 2 finished with value: 0.5531914893617021 and parameters: {'max_depth': 2, 'learning_rate': 0.059816326190312616, 'n_estimators': 199, 'min_child_weight': 9, 'lambda': 0.02106640268157479, 'alpha': 0.008695375907122877}. Best is trial 2 with value: 0.5531914893617021.

In [19]:
from dvclive.optuna import DVCLiveCallback

study.optimize(objective, n_trials = 32, callbacks = [DVCLiveCallback()])

[I 2023-07-22 08:01:53,447] Trial 0 finished with value: 0.5217391304347826 and parameters: {'max_depth': 1, 'learning_rate': 0.5826476619137811, 'n_estimators': 159, 'min_child_weight': 2, 'lambda': 0.19399192301428728, 'alpha': 0.039293080321705316}. Best is trial 0 with value: 0.5217391304347826.
	.DS_Store, demo-04-ChurnPrediction_HyperparameterTuningAndModelServing.ipynb, xgboost-hptuning-8172/dvc.yaml, xgboost-hptuning-3736/dvc.yaml, xgboost-hptuning-9783/dvc.yaml, xgboost-hptuning-4460/dvc.yaml, xgboost-hptuning-9991/dvc.yaml, xgboost-hptuning-3007/dvc.yaml, xgboost-hptuning-4474/dvc.yaml, xgboost-hptuning-7260/dvc.yaml, xgboost-hptuning-9628/dvc.yaml, xgboost-hptuning-2438/dvc.yaml, xgboost-hptuning-5086/dvc.yaml, xgboost-hptuning-3904/dvc.yaml, xgboost-hptuning-9262/dvc.yaml, xgboost-hptuning-7703/dvc.yaml, xgboost-hptuning-2481/dvc.yaml, xgboost-hptuning-2020/dvc.yaml, xgboost-hptuning-4977/dvc.yaml, xgboost-hptuning-9716/dvc.yaml, xgboost-hptuning-8068/dvc.yaml, xgboost-hptu

	.DS_Store, demo-04-ChurnPrediction_HyperparameterTuningAndModelServing.ipynb, .ipynb_checkpoints/demo-04-ChurnPrediction_HyperparameterTuningAndModelServing-checkpoint.ipynb, data/.DS_Store, data/.gitignore, data/Customertravel.csv.dvc
[I 2023-07-22 08:02:10,135] Trial 7 finished with value: 0.2222222222222222 and parameters: {'max_depth': 1, 'learning_rate': 0.031554797393411674, 'n_estimators': 109, 'min_child_weight': 1, 'lambda': 0.18536843617387433, 'alpha': 0.4812067071918032}. Best is trial 1 with value: 0.64.
	.DS_Store, demo-04-ChurnPrediction_HyperparameterTuningAndModelServing.ipynb, .ipynb_checkpoints/demo-04-ChurnPrediction_HyperparameterTuningAndModelServing-checkpoint.ipynb, data/.DS_Store, data/.gitignore, data/Customertravel.csv.dvc
[I 2023-07-22 08:02:11,271] Trial 8 finished with value: 0.6440677966101694 and parameters: {'max_depth': 12, 'learning_rate': 0.023858691251308923, 'n_estimators': 367, 'min_child_weight': 4, 'lambda': 0.45873221130970626, 'alpha': 0.0624

	.DS_Store, demo-04-ChurnPrediction_HyperparameterTuningAndModelServing.ipynb, .ipynb_checkpoints/demo-04-ChurnPrediction_HyperparameterTuningAndModelServing-checkpoint.ipynb, data/.DS_Store, data/.gitignore, data/Customertravel.csv.dvc
[I 2023-07-22 08:02:25,278] Trial 19 finished with value: 0.4 and parameters: {'max_depth': 7, 'learning_rate': 0.1241177748509562, 'n_estimators': 379, 'min_child_weight': 3, 'lambda': 0.008810665533414223, 'alpha': 9.91503756169069}. Best is trial 15 with value: 0.6666666666666667.
	.DS_Store, demo-04-ChurnPrediction_HyperparameterTuningAndModelServing.ipynb, .ipynb_checkpoints/demo-04-ChurnPrediction_HyperparameterTuningAndModelServing-checkpoint.ipynb, data/.DS_Store, data/.gitignore, data/Customertravel.csv.dvc
[I 2023-07-22 08:02:26,457] Trial 20 finished with value: 0.6538461538461539 and parameters: {'max_depth': 12, 'learning_rate': 0.19504048151193018, 'n_estimators': 309, 'min_child_weight': 8, 'lambda': 0.04385806131808751, 'alpha': 0.190649

	.DS_Store, demo-04-ChurnPrediction_HyperparameterTuningAndModelServing.ipynb, .ipynb_checkpoints/demo-04-ChurnPrediction_HyperparameterTuningAndModelServing-checkpoint.ipynb, data/.DS_Store, data/.gitignore, data/Customertravel.csv.dvc
[I 2023-07-22 08:02:39,761] Trial 31 finished with value: 0.6428571428571429 and parameters: {'max_depth': 14, 'learning_rate': 0.17688064889757654, 'n_estimators': 308, 'min_child_weight': 4, 'lambda': 0.005880153940348919, 'alpha': 0.09591313460177833}. Best is trial 25 with value: 0.6896551724137931.
	.DS_Store, demo-04-ChurnPrediction_HyperparameterTuningAndModelServing.ipynb, .ipynb_checkpoints/demo-04-ChurnPrediction_HyperparameterTuningAndModelServing-checkpoint.ipynb, data/.DS_Store, data/.gitignore, data/Customertravel.csv.dvc


# TODO Recording:

- Go to Studio and refresh the hyperparameter tuning project
- Show that there are 32 experiments there - we haven't yet dvc pushed the experiments so no metrics available
- Switch over to the terminal window and run this
- See the locall tracked metrics

`dvc exp show`

- Sort by the metric used to optimize i.e. f1 score

`dvc exp show --sort-by metric --sort-order desc`

- Push to DVC Studio

`dvc exp push origin --all-commits`

- Go to Studio immediately and keep clicking "Update Project" till we get all the experiments updated
- Select any two experiments and compare them
- Do comparison with two different experiments again

Obtaining the best set of hyperparameters

In [21]:
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 32
Best trial:
  Value: 0.6896551724137931
  Params: 
    max_depth: 8
    learning_rate: 0.096492494323816
    n_estimators: 323
    min_child_weight: 4
    lambda: 0.020677449421486978
    alpha: 0.13931875831001142


Now we’ll save the best parameters to a dictionary, and we’ll use the XGBClassifier function to create a new model with the best parameters and evaluating the performance

In [22]:
params = trial.params

In [23]:
xgb_model = XGBClassifier(**params)
    
pipe_xgb = Pipeline(steps = [('preprocessor', preprocessor), ('classifier', xgb_model )])
pipe_xgb.fit(X_train, y_train)

predictions = pipe_xgb.predict(X_test)

In [25]:
print('Test accuracy score: ', accuracy_score(y_test, predictions))
print('Test precision score: ', precision_score(y_test, predictions))
print('Test recall score: ', recall_score(y_test, predictions))
print('Test f1 score:', f1_score(y_test, predictions))

Test accuracy score:  0.8
Test precision score:  0.5882352941176471
Test recall score:  0.8333333333333334
Test f1 score: 0.6896551724137931


Now we will use MLEM for packaging the model for serving (based on MLEM docs, we are only serving the model here and not deploying it)

MLEM is a tool to easily package, deploy and serve Machine Learning models. It seamlessly supports a variety of scenarios like real-time serving and batch processing.

Saving machine learning models to files or loading them back into Python objects may seem like a simple task at first. For example, the pickle and torch libraries can serialize/deserialize model objects to/from files. However, MLEM adds some "special sauce" by inspecting the objects and saving their metadata into .mlem metafiles and using these intelligently later on.

However, when MLEM is used to save python model objects or enrich existing model files it adds its "special sauce" - all the things needed to reliably recreate this Python object later are extracted auto-magically and are written to a .mlem metadata file. This additional "metafile" is a yaml representation of the MLEM Object corresponding to the model Python object. This is why we refer to this operation as a "codification" of the model, and is a very powerful concept, underlining a lot of MLEM's abilities.

https://mlem.ai/doc/user-guide/basic-concepts

In [ ]:
pip install mlem

MLEM can analyze the code (or model files), and extract extra metadata which will become very useful when using the model.We can either save the model with MLEM to begin with, or use MLEM to import existing model file and save the metadata as a side effect.The sample_data argument will prove useful later when serving the model. MLEM uses it to infer the data schema of the payload automatically!

Alongside the model binary models/pipe_xgb, MLEM added the metadata file models/pipe_xgb.mlem. We refer to this as a "Codification" of the model.

In [27]:
from mlem.api import save

save(pipe_xgb, 'models/pipe_xgb', sample_data = X_test)

MlemModel(location=Location(path='/Users/loonycorn/projects/dvc/dvc_churn_prediction_hyperparameters/models/pipe_xgb.mlem', project=None, rev=None, uri='file:///Users/loonycorn/projects/dvc/dvc_churn_prediction_hyperparameters/models/pipe_xgb.mlem', project_uri=None, fs=<fsspec.implementations.local.LocalFileSystem object at 0x7fcc4eecc5e0>), params={}, artifacts={'data': LocalArtifact(uri='pipe_xgb', size=355708, hash='71596e845601d8c919bd48c839066b9c')}, requirements=Requirements(__root__=[InstallableRequirement(module='numpy', version='1.25.0', package_name=None, extra_index=None, source_url=None, vcs=None, vcs_commit=None), InstallableRequirement(module='pandas', version='1.5.3', package_name=None, extra_index=None, source_url=None, vcs=None, vcs_commit=None), InstallableRequirement(module='sklearn', version='1.2.1', package_name='scikit-learn', extra_index=None, source_url=None, vcs=None, vcs_commit=None), UnixPackageRequirement(package_name='libgomp1'), InstallableRequirement(mod

# TODO Recording:

- Go to the Finder window ~/projects/dvc/dvc_churn_prediction_hyperparameters
- Show the models/ folder has been created
- Open up the file pipe_xgb.mlem in Sublimetext and show

We are taking two instances from test data for getting predictions

In [28]:
predictions

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1])

In [29]:
X_test.iloc[3].values

array([30, 'No Record', 0, 3, 'Yes', 'No'], dtype=object)

In [30]:
X_test.iloc[12].values

array([37, 'Yes', 0, 3, 'Yes', 'Yes'], dtype=object)

You can either load and natively call the model object in your Python code anywhere, or use MLEM CLI for batch scoring against any dataset.

In [31]:
from mlem.api import load

model = load('models/pipe_xgb')  

features = [
    'Age', 'FrequentFlyer', 'AnnualIncomeClass', 
    'ServicesOpted', 'AccountSyncedToSocialMedia', 'BookedHotelOrNot'
]

sample_df = pd.DataFrame([
    [37, 'Yes', 0, 1, 'No', 'Yes'], 
    [34, 'Yes', 0, 0, 'Yes', 'No']
], columns = features)

y_pred = model.predict(sample_df)

print(y_pred)

[0 1]


# TODO Recording:

- Switch to the terminal window ~/projects/dvc/dvc_churn_prediction_hyperparameters
- Run the commands below to create a test.csv file (> is for overwrite, >> is for append)

`echo "Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot" > test.csv`

`echo "38,Yes,2,4,No,No" >> test.csv`

`echo "37,Yes,0,1,No,Yes" >> test.csv`

`cat test.csv`

# TODO Recording:

- Run this command below in the terminal, I've left it like this to preserve the formatting

In [33]:
mlem apply models/pipe_xgb test.csv \
    --method predict \
    --import \
    --import-type "pandas[csv]"    

⏳️ Importing object from test.csv
⏳️ Loading model from models/pipe_xgb.mlem
🍏 Applying `predict` method...
[1, 0]


### Notes:

MLEM can serve any model for you using different server implementations like FastAPI, RabbitMQ or Streamlit. Let's check out how it works with FastAPI since using it for serving is very common.

FastAPI is a modern, fast, and web framework for building APIs (Application Programming Interfaces) with Python. It is designed to be highly efficient, easy to use, and provide automatic validation, serialization, and documentation of APIs. FastAPI is built on top of Starlette, a high-performance asynchronous web framework, and Pydantic, a data validation and parsing library.

To expose your model to a external users via REST API, MLEM can use FastAPI to serve it.

To launch a FastAPI model server locally, simply run:

# TODO Recording

- Run the following commands in the terminal

`pip install mlem[fastapi]`

`mlem serve fastapi --model models/pipe_xgb --host 0.0.0.0 --port 8000`

- The server is now running, serving your model, and listening on http://0.0.0.0:8000 for HTTP requests
- Go to http://localhost:8000/docs and this is the Swagger UI for the Open API specifications
- Click on the GET request and execute it. 

### Notes:

#### Swagger UI

Swagger UI is a web-based graphical user interface that provides interactive documentation for APIs. It allows developers to visualize and explore the available API endpoints, understand their input parameters and response formats, and even test the API directly from the documentation page.

The Swagger UI is automatically generated based on the API's OpenAPI Specification (formerly known as Swagger Specification). OpenAPI Specification is a standardized format to describe RESTful APIs, and it defines the structure of the API, including its endpoints, request and response data, authentication methods, and more.

When an API is designed with OpenAPI Specification, tools like Swagger UI can read the specification and generate a user-friendly documentation page. The documentation is presented in a human-readable format with a graphical user interface, making it easy for developers to understand the API and interact with it.

#### GET
The response here gives us the interface of the API (predict and predictproba are the methods exposed and the return values defined)


# TODO Recording:

- Continue in the terminal window - take the request bodies from here (I've kept it here so the formatting is preserved)
- First we click on GET tab and then click on Try it out. Now in Post tab with predict Handler we can click on Try it out and Paste the below content inside the Request body and click on Execute to get the prediction
- Try out the curl command that they have provided in the Swagger API
- The first one's prediction should be 1, then 0
- **IMPORTANT**: With the same two request bodies try the command for the **predict_proba** function as well

In [ ]:
{
  "data": {
    "values": [
      {
        "": 0,
        "Age": 38,
        "FrequentFlyer": "Yes",
        "AnnualIncomeClass": 2,
        "ServicesOpted": 4,
        "AccountSyncedToSocialMedia": "No",
        "BookedHotelOrNot": "No"
      }
    ]
  }
}

In [ ]:
{
  "data": {
    "values": [
      {
        "": 0,
        "Age": 37,
        "FrequentFlyer": "Yes",
        "AnnualIncomeClass": 0,
        "ServicesOpted": 1,
        "AccountSyncedToSocialMedia": "No",
        "BookedHotelOrNot": "Yes"
      }
    ]
  }
}

### Now we will be Deploying models in production to  Docker

### Notes
Docker is an open-source platform that enables developers to automate the deployment, scaling, and management of applications inside lightweight, portable containers. Containers are a form of virtualization that packages an application and its dependencies into a single unit, ensuring consistency and reproducibility across different environments.

Here are some key concepts and features of Docker:

Containers: Containers are isolated environments that encapsulate an application along with its runtime, system tools, libraries, and settings. Each container runs as an isolated process on the host operating system, providing the benefits of virtualization without the overhead of running a full virtual machine.

Images: Docker uses images as templates for creating containers. An image is a read-only snapshot of a specific application and its dependencies. Images are built from a Dockerfile, which contains instructions for assembling the required components.

Dockerfile: A Dockerfile is a text file that defines the steps to build a Docker image. It includes commands for installing dependencies, copying files into the image, setting environment variables, and specifying the entry point for the container.

Registry: Docker provides a centralized repository called Docker Hub (hub.docker.com) where users can publish and share Docker images. However, private registries can also be used for secure storage and distribution of images.

# TODO Recording:

- Install Docker Desktop on your local machine, go to https://www.docker.com/products/docker-desktop/
- Note that it detects what kind of machine you are on and presents the right download
- Download and install this
- Click on the Docker icon on the top right and show the Dashboard
- Click on Containers and Images from the left navigation pane

- Go to the terminal and run

`docker --version`

- This is the command to view running containers (should be empty)

`docker ps`

- This is the command to view the Docker images that we have locally (should be empty)

`docker images`

- Install the mlem library to deploy to Docker

`pip install mlem[docker]`

### Notes:


Building is a way to “bake” your model into something usable in production like a Docker image, or export your model into another format or even export the underlying requirements and dependencies of the model, allowing one to create virtual environments out of it.

##### 3 steps for deployment

- Create a builder (this is just an MLEM object i.e. an object with metadata in a .mlem file) that will define how your model will be built. In this case of Docker this declaration basically defines all things you need to build a docker image. It includes image name, what server you want to serve your model with, and optionally some other stuff, `mlem declare` will preconfigure your builder with a yaml file

- Use the builder as a value for --load option in mlem build. This is what you use to build models into re-usable assets you can distribute and use in production, such as a Docker image or Python package.

- Deploy the asset representing your model (i.e. the Docker image) using `mlem deployment`. The deployment commands are used to manage the lifecycle of deployments along with giving access to methods of the deployed model. A "deployment" is an application/service instance consisting of a server, serving a specific model, using a specific environment definition, and running on a target platform.

IMPORTANT NOTE:

If you run Step #3 directly that will actually automatically perform step #1 and step #2. We just show each step so it's clear what is happening

mlem deploy run docker_container docker_app.mlem --model models/pipe_xgb --server fastapi --ports.0 8080:8080

mlem deployment run will create 2 auxiliary files:

- {deployment}.mlem - A deployment spec (or declaration). Reusing this app declaration you can, for example, re-deploy the same application with a different model.

- {deployment}.mlem.state - A state file containing more information about the deployment, including which MLEM model we used, and the URL of the deployment. Each deployment has state, which is a snapshot of the actual state of your deployment. It is created and updated by MLEM during the deployment process to keep track of parameters needed for state management, and is stored separately from the declaration. 

# TODO Recording:

- Run the following commands on the terminal

`mkdir serving`

`cd serving`

`pip install mlem[docker]`

`mlem declare builder docker docker_builder.mlem --image.name mlem-model --daemon.host "" --server fastapi`

- A builder file is created docker_builder.mlem

`ls -l`

- Show the file this declaration basically defines all things you need to build a docker image. It includes image name, what server you want to serve your model with

`cat docker_builder.mlem`

- Build the Docker image. An image is a read-only snapshot of a specific application and its dependencies. Images are built from a Dockerfile, which contains instructions for assembling the required components. The Dockerfile is automatically generated by MLEM - we'll see it in a bit.

`mlem build --load docker_builder.mlem --model models/pipe_xgb`

- Go to the Docker Desktop Dashboard and click on "Images" on the left - show that a new Docker image has been created
- Click on Containers and there are no running containers yet
- Come back to the terminal and continue

`mlem deploy run docker_container docker_app.mlem --model models/pipe_xgb --server fastapi --ports.0 8080:8080`

- This will use the prebuilt container and run your server inside Docker
- This generates a few files

- This is the MLEM Object for the deployment - defines the docker deployment

`cat docker_app.mlem`

- 

- Go to the Docker Desktop Dashboard and click on "Containers" on the left - a new container is running
- Click throught to the Container here and click on the tabs and show Logs, Inspect, Terminal, Files, and Stats
- Go to Terminal (this is actually a terminal to the shell running inside)
- Run the following commands

- See the files needed to run our server

`ls -l`

- This is the MLEM Object representing our model

`cat model.mlem`

- Need to build the environment needed by the model

`cat requirements.txt`

- Defines the components that make up the container
`cat Dockerfile`

- Serves the model and model configuration
`cat run.sh`
`cat server.yaml`

#### Getting predictions

- IMPORTANT Please following the same steps for getting predictions as earlier
- Can do one POST for predict() and one for predict_proba()
- No need to do curl

#### Deployment management

`mlem deployment status docker_app.mlem`

`mlem deployment remove docker_app.mlem`

- No running containers

`docker ps`